In [1]:
!pip install pulp
!pip install folium
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value
import pandas as pd
import folium
from branca.element import Div

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 45.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# Dados das fábricas e dos CDs
fabricas = {
    'F1': {'lat': -25.243616, 'lon': -57.542389, 'capacidade': 8500000, 'pais': 'PY'},
    'F2': {'lat': -25.357959, 'lon': -57.481983, 'capacidade': 9200000, 'pais': 'PY'},
    'F3': {'lat': -27.114697, 'lon': -52.691429, 'capacidade': 10000000, 'pais': 'BR'},
    'F4': {'lat': -27.102029, 'lon': -54.484598, 'capacidade': 8800000, 'pais': 'AR'},
    'F5': {'lat': -30.903511, 'lon': -56.927418, 'capacidade': 9000000, 'pais': 'UY'},
}

cds = {
    'CD1': {'lat': -25.24007, 'lon': -57.451505, 'demanda': 1314797, 'pais': 'PY'},
    'CD2': {'lat': -25.28263, 'lon': -57.656959, 'demanda': 301760, 'pais': 'PY'},
    'CD3': {'lat': -25.46950, 'lon': -54.515057, 'demanda': 462778, 'pais': 'BR'},
    'CD4': {'lat': -26.38708, 'lon': -54.657338, 'demanda': 1308778, 'pais': 'AR'},
    'CD5': {'lat': -27.87519, 'lon': -54.452076, 'demanda': 930634, 'pais': 'BR'},
    'CD6': {'lat': -30.45454, 'lon': -56.843342, 'demanda': 764849, 'pais': 'UY'},
    'CD7': {'lat': -24.96514, 'lon': -53.337918, 'demanda': 1509150, 'pais': 'BR'},
    'CD8': {'lat': -27.07077, 'lon': -52.638958, 'demanda': 160456, 'pais': 'BR'},
    'CD9': {'lat': -26.16875, 'lon': -58.190166, 'demanda': 231122, 'pais': 'AR'},
    'CD10': {'lat': -29.72809, 'lon': -56.930145, 'demanda': 1192082, 'pais': 'BR'},
    'CD11': {'lat': -31.32757, 'lon': -54.062244, 'demanda': 544315, 'pais': 'BR'},
    'CD12': {'lat': -31.10517, 'lon': -55.636963, 'demanda': 845639, 'pais': 'UY'},
    'CD13': {'lat': -30.93391, 'lon': -57.532847, 'demanda': 1218029, 'pais': 'UY'},
    'CD14': {'lat': -30.99506, 'lon': -57.766227, 'demanda': 1171484, 'pais': 'UY'},
    'CD15': {'lat': -31.36196, 'lon': -58.032099, 'demanda': 501241, 'pais': 'AR'},
    'CD16': {'lat': -23.70115, 'lon': -54.642188, 'demanda': 314710, 'pais': 'BR'},
    'CD17': {'lat': -27.46327, 'lon': -60.51042, 'demanda': 1577345, 'pais': 'AR'},
    'CD18': {'lat': -26.11752, 'lon': -60.278769, 'demanda': 539155, 'pais': 'AR'},
    'CD19': {'lat': -27.92988, 'lon': -58.095279, 'demanda': 965912, 'pais': 'AR'},
    'CD20': {'lat': -28.58554, 'lon': -59.832146, 'demanda': 930688, 'pais': 'AR'}
}




    # Dados do custo de transporte por km por tonelada de produto
custo_transporte = [
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
]
# Dados das taxas alfandegárias por tonelada de produto
#taxas_alfandegarias = {
#    'F1': {'PY': 3.35},
#    'F2': {'PY': 3.35},
#    'F3': {'BR': 3.5},
#    'F4': {'AR': 2.45},
#    'F5': {'UY': 2.2},
#}

# Dados das distâncias entre fábricas e CDs
distancias = [
    [13, 17, 659, 538, 816, 336, 324, 346, 226, 794, 436, 424, 288, 112, 681, 593, 577, 258, 135, 512],
    [14, 21, 672, 552, 818, 775, 760, 698, 599, 75, 479, 466, 335, 352, 937, 663, 651, 10, 256, 731],
    [148, 163, 810, 586, 748, 815, 799, 615, 464, 199, 922, 337, 605, 588, 337, 830, 814, 646, 588, 151],
    [817, 801, 796, 648, 94, 815, 16, 665, 672, 149, 839, 855, 868, 717, 172, 445, 457, 533, 486, 1055],
    [646, 661, 153, 578, 326, 410, 426, 959, 737, 797, 445, 461, 694, 458, 543, 519, 535, 950, 728, 788]
]




taxas_alfandegarias = {
    "PY-PY": 0,
    "PY-BR": 2.2,
    "PY-AR": 3.35,
    "PY-UY": 5.55,
    "BR-PY": 1.8,
    "BR-BR": 0,
    "BR-AR": 3.5,
    "BR-UY": 2.25,
    "AR-PY": 2.3,
    "AR-BR": 3.5,
    "AR-AR": 0,
    "AR-UY": 2.2,
    "UY-PY": 4.75,
    "UY-BR": 3.15,
    "UY-AR": 2.45,
    "UY-UY": 0
}


In [21]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus, LpBinary, LpInteger

# Criar o problema de otimização
prob = LpProblem("Minimizar_custo", LpMinimize)

# Variáveis de decisão
x = LpVariable.dicts("x", ((i, j) for i in range(len(fabricas)) for j in range(len(cds))), cat=LpInteger, lowBound=0)
y = LpVariable.dicts("y", range(len(fabricas)), cat=LpBinary)

# Função objetivo
# Função objetivo
custo_transporte_total = lpSum([x[i, j] * custo_transporte[i][j] * distancias[i][j] for i in range(len(fabricas)) for j in range(len(cds))])
custo_taxas_total = lpSum([taxas_alfandegarias.get(f"{fabricas[f'F{i+1}']['pais']}-{cds[f'CD{j+1}']['pais']}", 0) * x[i, j] for i in range(len(fabricas)) for j in range(len(cds)) if fabricas[f'F{i+1}']['pais'] != cds[f'CD{j+1}']['pais']])

custo_total = custo_transporte_total + custo_taxas_total
prob += custo_total


# Restrições
for j in range(len(cds)):
    prob += lpSum([x[i, j] for i in range(len(fabricas))]) == cds[f'CD{j+1}']['demanda']

for i in range(len(fabricas)):
    prob += lpSum([x[i, j] for j in range(len(cds))]) <= fabricas[f'F{i+1}']['capacidade'] * y[i]

prob += lpSum(y[i] for i in range(len(fabricas))) == 2

# Resolver o problema
prob.solve()

print("Status:", LpStatus[prob.status])

# Imprimir resultados
for i in range(len(fabricas)):
    print(f"Fabrica F{i+1}: {y[i].value()}")

for i in range(len(fabricas)):
    for j in range(len(cds)):
        print(f"x({i+1}, {j+1}): {x[i, j].value()}")


# Criar uma lista com os dados das quantidades enviadas
data = []
for i in range(len(fabricas)):
    for j in range(len(cds)):
        if x[i, j].value() > 0:
            data.append([f'Fábrica {i+1}', f'CD {j+1}', x[i, j].value()])

# Criar um DataFrame pandas com os dados
df = pd.DataFrame(data, columns=['Fábrica', 'CD', 'Quantidade Enviada (toneladas)'])

# Exibir a tabela
df

,Fábrica,CD,Quantidade Enviada (toneladas)
0,Fábrica 1,CD 1,1314797.0
1,Fábrica 1,CD 2,301760.0
2,Fábrica 1,CD 6,764849.0
3,Fábrica 1,CD 8,160456.0
4,Fábrica 1,CD 9,231122.0
5,Fábrica 1,CD 11,544315.0
6,Fábrica 1,CD 12,845639.0
7,Fábrica 1,CD 13,1218029.0
8,Fábrica 1,CD 14,1171484.0
9,Fábrica 1,CD 18,50949.0


In [22]:
# Criar mapa
m = folium.Map(location=[-27, -55], zoom_start=5)

# Adicionar marcadores das fábricas e CDs
for i in range(len(fabricas)):
    if y[i].value() == 1:
        folium.Marker(
            location=[fabricas[f'F{i+1}']['lat'], fabricas[f'F{i+1}']['lon']],
            icon=folium.Icon(color="red"),
            popup=f"Fábrica {i+1}",
        ).add_to(m)

    for j in range(len(cds)):
        if x[i, j].value() > 0:
            cd_color = "blue"
            if i == 0:  # Fábrica 1
                cd_color = "green"
            elif i == 3:  # Fábrica 4
                cd_color = "blue"
                
            folium.Marker(
                location=[cds[f'CD{j+1}']['lat'], cds[f'CD{j+1}']['lon']],
                icon=folium.Icon(color=cd_color),
                popup=f"CD {j+1}: Demanda {cds[f'CD{j+1}']['demanda']} toneladas",
            ).add_to(m)

display(m)
